In [1]:
import modin.pandas as pd
import nums
from nums import numpy as nps

from nums.experimental.nums_modin import from_modin

import warnings
warnings.filterwarnings("ignore");

nums.init()

2021-11-01 20:48:19,213	INFO services.py:1265 -- View the Ray dashboard at http://127.0.0.1:8265


Using driver node ip as head node.
head node 10.0.0.4
total cpus 30.0
device_grid (0, 0) 0=node:10.0.0.4/cpu:1


In [2]:
def rmse(actual, expected):
    import numpy as np
    if type(actual) == type(nps.array([])):
        return nps.sqrt(nps.mean((expected - actual) ** 2)).get()
    elif type(actual) == type(np.array([])):
        return np.sqrt(np.mean((expected - actual) ** 2))    

In [5]:
%%time
X_data = pd.DataFrame()
y_data = pd.DataFrame()

for year in range(2019, 2021):
    X_data = X_data.append(pd.read_csv("data/X_" + str(year) + ".csv.gz", compression='gzip'))
    y_data = y_data.append(pd.read_csv("data/y_bin_" + str(year) + ".csv.gz", compression='gzip'))

CPU times: user 5.21 s, sys: 628 ms, total: 5.84 s
Wall time: 8.01 s


In [6]:
X_data.shape

(6755037, 8)

In [7]:
y_data.shape

(6755037, 1)

In [8]:
X = from_modin(X_data).astype(float)
y = from_modin(y_data).astype(float).reshape(-1)

In [9]:
shuffle = nps.random.permutation(X.shape[0])
X = X[shuffle]
y = y[shuffle]
split = int(X.shape[0] * 0.8)
X_train = X[:split]
X_test = X[split:]
y_train = y[:split]
y_test = y[split:]

In [10]:
from nums.models.glms import LogisticRegression

model = LogisticRegression(solver="newton", penalty="l2", tol=1e-4)

In [11]:
%%time
model.fit(X_train, y_train)

CPU times: user 5.56 s, sys: 2.4 s, total: 7.96 s
Wall time: 9.8 s


In [12]:
%%time
training_results = model.predict(X_train)

CPU times: user 117 ms, sys: 39.8 ms, total: 157 ms
Wall time: 62.4 ms


In [13]:
%%time
test_results = model.predict(X_test)

CPU times: user 35.9 ms, sys: 12.5 ms, total: 48.4 ms
Wall time: 17.5 ms


In [14]:
print("Training accuracy: ", nps.sum(training_results == y_train).get() / y_train.shape[0])
print("Testing accuracy:  ", nps.sum(test_results == y_test).get() / y_test.shape[0])

Training accuracy:  0.7101495939418534
Testing accuracy:   0.7105228096354722


In [15]:
X_train_np = X_train.get()
X_test_np = X_test.get()
y_train_np = y_train.get()
y_test_np = y_test.get()

In [16]:
from sklearn.linear_model import LogisticRegression

In [17]:
model = LogisticRegression()

In [18]:
%%time
model.fit(X_train_np, y_train_np)

CPU times: user 11min 12s, sys: 9min 26s, total: 20min 38s
Wall time: 3min 31s


LogisticRegression()

In [19]:
%%time
training_results = model.predict(X_train_np)

CPU times: user 3.93 s, sys: 4.09 s, total: 8.02 s
Wall time: 279 ms


In [20]:
%%time
test_results = model.predict(X_test_np)

CPU times: user 2.35 s, sys: 3.52 s, total: 5.86 s
Wall time: 100 ms


In [21]:
print("Training accuracy: ", model.score(X_train_np, y_train_np))
print("Testing accuracy:  ", model.score(X_test_np, y_test_np))

Training accuracy:  0.7053674212333058
Testing accuracy:   0.7058440808640659
